In [29]:
import signals.futures_signals as fs
import signals.utils as su
signals_output = fs.get_futures_butterfly_signals(ticker_list=['LNJ2016','LNN2016','LNV2016'],date_to=20160114)

price_ratio = signals_output['price_ratio']
price_ratio

array([ 0.86200964,  0.85750766,  0.8602261 ,  0.85745394,  0.85608856,
        0.85531629,  0.85361345,  0.85719178,  0.85223426,  0.85784636,
        0.85960265,  0.86857905,  0.86705491,  0.86693548,  0.86248753,
        0.86350099,  0.86252479,  0.86559843,  0.86420157,  0.8575172 ,
        0.85272965,  0.83825481,  0.83829365,  0.83783333,  0.84105298,
        0.83927404,  0.84165593,  0.84314039,  0.84382022,  0.84532605,
        0.84489403,  0.84512705,  0.84705317,  0.84937238,  0.85191808,
        0.84986988,  0.8516537 ,  0.86154835,  0.86219081,  0.86453674])

In [1]:
import opportunity_constructs.futures_butterfly as fb
fb.generate_futures_butterfly_sheet_4date(date_to=20160114)



{'butterflies':      ticker1  ticker2  ticker3 tickerHead tickerClass  trDte1  trDte2  trDte3  \
 0     BK2016   BM2016   BN2016          B      Energy      54      75      96   
 1     BM2016   BN2016   BQ2016          B      Energy      75      96     118   
 2     BN2016   BQ2016   BU2016          B      Energy      96     118     138   
 3     BQ2016   BU2016   BV2016          B      Energy     118     138     160   
 4     BU2016   BV2016   BX2016          B      Energy     138     160     182   
 5     BV2016   BX2016   BZ2016          B      Energy     160     182     203   
 6     BX2016   BZ2016   BF2017          B      Energy     182     203     224   
 7     BZ2016   BF2017   BG2017          B      Energy     203     224     244   
 8     BF2017   BG2017   BH2017          B      Energy     224     244     266   
 9     BG2017   BH2017   BJ2017          B      Energy     244     266     286   
 10    BH2017   BJ2017   BK2017          B      Energy     266     286     309   
 

In [22]:
import opportunity_constructs.spread_carry as sc

#spread_carry_output = sc.generate_spread_carry_sheet_4date(report_date=20160210)
#spread_report = spread_carry_output['spread_report']
#ticker_head_list = list(spread_report['tickerHead'].unique())
# Compare BO and S sizing

spread_report[spread_report['tickerHead']==ticker_head_list[19]]
#spread_report[spread_report['tickerHead']=='S']

,ticker1,ticker2,tickerHead,front_tr_dte,carry,q_carry,reward_risk,price,q,upside,downside,change5,change10,change20
0,KCH2016,KCK2016,KC,26,NaN,NaN,NaN,-2.10,33,480.00,-245.63,NaN,NaN,NaN
1,KCK2016,KCN2016,KC,68,-56.25,-5,-0.08,-1.95,38,86.16,-93.75,NaN,NaN,NaN
2,KCN2016,KCU2016,KC,110,-37.50,1,-0.04,-1.85,37,103.12,-232.13,NaN,NaN,NaN
3,KCU2016,KCZ2016,KC,154,168.75,-7,0.07,-2.30,44,192.84,-283.03,NaN,NaN,NaN
4,KCZ2016,KCH2017,KC,217,-37.50,1,-0.01,-2.20,43,200.44,-277.22,NaN,NaN,NaN


In [2]:
import formats.strategy_followup_formats as sff
import numpy as np
import pandas as pd
writer_out = sff.generate_futures_butterfly_followup_report()
sff.generate_spread_carry_followup_report(writer=writer_out)

In [4]:
gfp.get_futures_price_preloaded(ticker='BOZ2016',settle_date=pd.to_datetime(settle_date))

,settle_date,ticker,ticker_head,ticker_month,ticker_year,cal_dte,tr_dte,close_price,volume,cont_indx,change1,change2,change5,change10,change20,change_5,change_1
122852,2016-01-25,BOZ2016,BO,12,2016,324,226,30.91,2983,201612,0.18,0.05,NaN,NaN,NaN,0.67,-0.08


In [1]:
import my_sql_routines.my_sql_utilities as msu
con = msu.get_my_sql_connection()
import formats.strategy_followup_formats as sff
import ta.prepare_daily as prep
import contract_utilities.expiration as exp
report_date = exp.doubledate_shift_bus_days()


writer_out = sff.generate_futures_butterfly_followup_report(as_of_date=report_date, con=con)
sff.generate_spread_carry_followup_report(as_of_date=report_date, con=con, writer=writer_out)
prep.move_from_dated_folder_2daily_folder(ext='ta', file_name='followup', folder_date=report_date)


con.close()

In [1]:
import my_sql_routines.my_sql_utilities as msu
con = msu.get_my_sql_connection()
import formats.strategy_followup_formats as sff
import ta.prepare_daily as prep
import contract_utilities.expiration as exp
report_date = exp.doubledate_shift_bus_days()

In [1]:
import shared.directory_names as dn
import contract_utilities.expiration as exp
import pandas as pd
as_of_date = exp.doubledate_shift_bus_days()
import formats.strategy_followup_formats as sff
import my_sql_routines.my_sql_utilities as msu
con = msu.get_my_sql_connection()

ta_output_dir = dn.get_dated_directory_extension(folder_date=as_of_date, ext='ta')

writer = pd.ExcelWriter(ta_output_dir + '/followup.xlsx', engine='xlsxwriter')

sff.generate_futures_butterfly_followup_report(as_of_date=as_of_date, con=con,writer=writer)

excell done! 


In [1]:
import formats.futures_strategy_formats as fsf
import formats.strategy_followup_formats as sff
import formats.risk_pnl_formats as rpf
import my_sql_routines.futures_price_loader as fpl
import my_sql_routines.my_sql_utilities as msu
import get_price.presave_price as pp
import opportunity_constructs.futures_butterfly as fb
import contract_utilities.expiration as exp
import ta.prepare_daily as prep

con = msu.get_my_sql_connection()
report_date = exp.doubledate_shift_bus_days()

rpf.generate_portfolio_pnl_report(as_of_date=report_date, con=con)
prep.move_from_dated_folder_2daily_folder(ext='ta', file_name='pnl', folder_date=report_date)




In [1]:

import read_exchange_files.read_cme_files as rcf
import pandas as pd
data_read_out = rcf.read_cme_settle_txt_files()
    
decoded_data = data_read_out['decoded_data']     
data_start_list = data_read_out['data_start_list']     
data_end_list = data_read_out['data_end_list']             
title_list = data_read_out['title_list']     
data_list = data_read_out['data_list']   
volume_list = data_read_out['volume_list']  
settle_list = data_read_out['settle_list'] 
month_strike_list = data_read_out['month_strike_list']  
filter_1_list = data_read_out['filter_1_list']  
filter_2_list = data_read_out['filter_2_list']  
total_volume_list = data_read_out['total_volume_list']  


summary_data = pd.DataFrame.from_items([('title',title_list),('volume',total_volume_list)])
summary_data[summary_data['volume']>0]

summary_sorted = summary_data.sort('volume',ascending=False)
summary_sorted

,title,volume
6,C Corn Futures,263082
64,S Soybean Futures,197407
74,W Wheat Futures,185499
4,BO Soybean Oil Futures,97921
68,SM Soybean Meal Futures,94883
31,KEF Kansas City Wheat Futures,56749
18,FCP BMD Crude Palm Oil Future,43237
37,LC CME LIVE CATTLE FUTURES,43103
39,LH CME LEAN HOG FUTURES,41996
79,ZCT Corn TAS Futures,24286


In [2]:


import read_exchange_files.read_cme_files as rcf

[rcf.process_title(x) for x in summary_sorted['title']]

C Corn Futures
S Soybean Futures
W Wheat Futures
BO Soybean Oil Futures
SM Soybean Meal Futures
KEF Kansas City Wheat Futures
FCP BMD Crude Palm Oil Future
LC CME LIVE CATTLE FUTURES
LH CME LEAN HOG FUTURES
ZCT Corn TAS Futures
PY MAR16 Corn Options PUT
CZO MAR16 SOYBEANS OPTION PUT
MWE Minneapolis Hard Red Spring Wheat Futures
FC CME FEEDER CATTLE FUTURES
CZO MAR16 SOYBEANS OPTION CALL
CZO MAY16 SOYBEANS OPTION CALL
ZWT Wheat TAS Futures
KLI BMD-KUALA LUMPAR INDEX FUTURE
WZ MAR16 Wheat Options CALL
PY DEC16 Corn Options PUT
PY MAR16 Corn Options CALL
ZLT Soybean Oil TAS Futures
PY JLY16 Corn Options CALL
PY APR16 Corn Options CALL
PY DEC16 Corn Options CALL
PY JLY16 Corn Options PUT
SBT Soybean TAS Futures
PY SEP16 Corn Options PUT
WZ APR16 Wheat Options CALL
CZO JLY16 SOYBEANS OPTION PUT
PY MAY16 Corn Options PUT
CZO NOV16 SOYBEANS OPTION CALL
OH APR16 LEAN HOG OPTIONS PUT
OK APR16 LIVE CATTLE OPTIONS CALL
MY MAR16 Soybean Meal Options CALL
ZMT Soybean Meal TAS Futures
WZ MAY16 Wheat

[{'maturity_string': [], 'option_type': '', 'ticker_head': 'C'},
 {'maturity_string': [], 'option_type': '', 'ticker_head': ''},
 {'maturity_string': [], 'option_type': '', 'ticker_head': 'W'},
 {'maturity_string': [], 'option_type': '', 'ticker_head': 'BO'},
 {'maturity_string': [], 'option_type': '', 'ticker_head': 'SM'},
 {'maturity_string': [], 'option_type': '', 'ticker_head': 'W'},
 {'maturity_string': [], 'option_type': '', 'ticker_head': ''},
 {'maturity_string': [], 'option_type': '', 'ticker_head': 'LC'},
 {'maturity_string': [], 'option_type': '', 'ticker_head': 'LN'},
 {'maturity_string': [], 'option_type': '', 'ticker_head': 'C'},
 {'maturity_string': 'MAR16', 'option_type': 'P', 'ticker_head': 'C'},
 {'maturity_string': 'MAR16', 'option_type': 'P', 'ticker_head': 'S'},
 {'maturity_string': [], 'option_type': '', 'ticker_head': 'W'},
 {'maturity_string': [], 'option_type': '', 'ticker_head': 'FC'},
 {'maturity_string': 'MAR16', 'option_type': 'C', 'ticker_head': 'S'},
 {'m

In [24]:

import contract_utilities.expiration as exp
report_date = exp.doubledate_shift_bus_days()
import futures_charts.futures_curve as fc
bf_signals_output = fc.get_butterfly_scatter_plot(report_date=report_date,id=169)

In [25]:
aligned_output = bf_signals_output['aligned_output']
aligned_data = aligned_output['aligned_data']
bf_price = aligned_data['c1']['close_price']+aligned_data['c3']['close_price']-2*aligned_data['c2']['close_price']

In [31]:
import matplotlib.pyplot as plt
plt.plot(bf_price)
plt.grid()         
         
plt.show()

In [29]:
bad_data = aligned_data[bf_price<-0.004]   #<-0.03
bad_data

# HOV2007,HOX2007, HOZ2007 20070213 to 20070413

c1                                                      \
              ticker ticker_head ticker_month ticker_year cal_dte tr_dte   
settle_date                                                                
2006-02-15   HOV2006          HO           10        2006     226    157   
2006-02-16   HOV2006          HO           10        2006     225    156   
2006-02-17   HOV2006          HO           10        2006     224    155   
2006-02-21   HOV2006          HO           10        2006     220    154   
2006-02-22   HOV2006          HO           10        2006     219    153   
2006-02-23   HOV2006          HO           10        2006     218    152   
2006-02-24   HOV2006          HO           10        2006     217    151   
2006-02-27   HOV2006          HO           10        2006     214    150   
2006-02-28   HOV2006          HO           10        2006     213    149   
2006-03-01   HOV2006          HO           10        2006     212    148   
2006-03-02   HOV2006          HO           10        2006     211    147   
2006-03-03   HOV2006          HO           10        2006     210    146   
2006-03-06   HOV2006          HO           10        2006     207    145   
2006-03-07   HOV2006          HO           10        2006     206    144   
2006-03-08   HOV2006          HO           10        2006     205    143   
2006-03-09   HOV2006          HO           10        2006     204    142   
2006-03-10   HOV2006          HO           10        2006     203    141   
2006-03-13   HOV2006          HO           10        2006     200    140   
2006-03-14   HOV2006          HO           10        2006     199    139   
2006-03-15   HOV2006          HO           10        2006     198    138   
2006-03-16   HOV2006          HO           10        2006     197    137   
2006-03-17   HOV2006          HO           10        2006     196    136   
2006-03-20   HOV2006          HO           10        2006     193    135   
2006-03-21   HOV2006          HO           10        2006     192    134   
2006-03-22   HOV2006          HO           10        2006     191    133   
2006-03-24   HOV2006          HO           10        2006     189    131   
2007-02-13   HOV2007          HO           10        2007     227    158   
2007-02-28   HOV2007          HO           10        2007     212    148   
2007-03-16   HOV2007          HO           10        2007     196    136   
2007-03-23   HOV2007          HO           10        2007     189    131   
2007-03-30   HOV2007          HO           10        2007     182    126   
2007-04-05   HOV2007          HO           10        2007     176    122   
2007-04-13   HOV2007          HO           10        2007     168    117   
2008-02-01   HOV2008          HO           10        2008     242    167   
2008-02-04   HOV2008          HO           10        2008     239    166   
2008-02-05   HOV2008          HO           10        2008     238    165   
2008-02-06   HOV2008          HO           10        2008     237    164   
2008-12-04   HOV2009          HO           10        2009     300    206   
2008-12-05   HOV2009          HO           10        2009     299    205   
2008-12-08   HOV2009          HO           10        2009     296    204   
2008-12-10   HOV2009          HO           10        2009     294    202   

                                                       ...            c3  \
            close_price volume cont_indx change1       ...       change5   
settle_date                                            ...                 
2006-02-15       1.7848    167    200610  0.0262       ...        0.0681   
2006-02-16       1.7995     67    200610  0.0147       ...        0.0052   
2006-02-17       1.8257     12    200610 -0.0233       ...        0.0097   
2006-02-21       1.8404     56    200610  0.0006       ...        0.0475   
2006-02-22       1.8171    201    200610  0.0559       ...        0.0942   
2006-02-23       1.8177    170    200610 -0.0397       ...       

In [21]:
aligned_data.iloc[1047]

c1                 ticker                      HOV2007
                   ticker_head                      HO
                   ticker_month                     10
                   ticker_year                    2007
                   cal_dte                         203
                   tr_dte                          141
                   close_price                  1.8272
                   volume                          119
                   cont_indx                    200710
                   change1                       0.014
                   change2                      0.0196
                   change5                      0.0075
                   change10                     0.0835
                   change20                     0.1669
                   change_5                    -0.0345
                   change_1                    -0.0407
c2                 ticker                      HOX2007
                   ticker_head                      HO
          

In [1]:
import shared.directory_names as dn
options_data_dir = dn.get_dated_directory_extension(folder_date=20160208,ext='comex_options_csv_address')
options_data_dir

'C:\\Research/data/options_data_raw/2016/201602/20160208'

In [3]:
import contract_utilities.expiration as exp
report_date = exp.doubledate_shift_bus_days()
report_date

20160208

In [4]:
import scheduled_jobs.options_data_load_job as odlj
import shared.downloads as sd
sd.download_csv_from_web(web_address=odlj.comex_options_csv_address)


,PRODUCT SYMBOL,CONTRACT MONTH,CONTRACT YEAR,CONTRACT DAY,PUT/CALL,STRIKE,CONTRACT,PRODUCT DESCRIPTION,OPEN,HIGH,...,LOW AB INDICATOR,LAST,LAST AB INDICATOR,SETTLE,PT CHG,EST. VOL,PRIOR SETTLE,PRIOR VOL,PRIOR INT,TRADEDATE
0,HX,03,2016,,C,0.25,HXH16 C025,Mar16 Copper Option Call,,,...,,,,1.8405,-.0125,0,1.8530,,,02/08/2016
1,HX,03,2016,,C,0.5,HXH16 C050,Mar16 Copper Option Call,,,...,,,,1.5905,-.0125,0,1.6030,,,02/08/2016
2,HX,03,2016,,C,0.75,HXH16 C075,Mar16 Copper Option Call,,,...,,,,1.3405,-.0125,0,1.3530,,,02/08/2016
3,HX,03,2016,,C,1,HXH16 C100,Mar16 Copper Option Call,,,...,,,,1.0905,-.0125,0,1.1030,,,02/08/2016
4,HX,03,2016,,C,1.05,HXH16 C105,Mar16 Copper Option Call,,,...,,,,1.0405,-.0125,0,1.0530,,,02/08/2016
5,HX,03,2016,,C,1.1,HXH16 C110,Mar16 Copper Option Call,,,...,,,,.9905,-.0125,0,1.0030,,,02/08/2016
6,HX,03,2016,,C,1.15,HXH16 C115,Mar16 Copper Option Call,,,...,,,,.9405,-.0125,0,.9530,,,02/08/2016
7,HX,03,2016,,C,1.2,HXH16 C120,Mar16 Copper Option Call,,,...,,,,.8905,-.0125,0,.9030,,,02/08/2016
8,HX,03,2016,,C,1.25,HXH16 C125,Mar16 Copper Option Call,,,...,,,,.8405,-.0125,0,.8530,,,02/08/2016
9,HX,03,2016,,C,1.3,HXH16 C130,Mar16 Copper Option Call,,,...,,,,.7905,-.0125,0,.8030,,,02/08/2016


In [1]:
import scheduled_jobs.options_data_load_job

In [41]:
with open(r'C:\Research\data\options_data_raw\2016\201602\20160208\interest_rate.pkl','rb') as handle:
    wuhu = pickle.load(handle)

In [ ]:
scatter_matrix = np.zeros((3,3))

for i in range(all_samples.shape[1]):
    scatter_matrix +=(all_samples[:,i].reshape(3,1)).dot(
    (all_samples[:,i].reshape(3,1)-mean_vector).T)
print('Scatter Matrix\n',scatter_matrix)    

In [ ]:
cov_mat = np.cov([all_samples[0,:], all_samples[1,:],all_samples[2,:]])
print('Covariance Matrix:\n', cov_mat)

In [ ]:
for i in range(len(eig_val_sc)):
    eigv = eig_vec_sc[:,i].reshape(1,3).T
    np.testing.assert_array_almost_equal(scatter_matrix.dot(eigv),
                                         eig_val_sc[i] * eigv,
                                         decimal=6, err_msg='', verbose=True)

In [ ]:
transformed = matrix_w.T.dot(all_samples)
assert transformed.shape == (2,40), "The matrix is not 2x40 dimensional."
transformed
matrix_w

In [ ]:
import matplotlib.pyplot as plt
plt.plot(merged_dataframe.index,(merged_dataframe['BOH2016']['close_price']+merged_dataframe['BON2016']['close_price'])/2-
         merged_dataframe['BOK2016']['close_price'])
         
         
plt.show()

In [ ]:
(merged_dataframe['BOH2016']['close_price']+merged_dataframe['BON2016']['close_price'])/2-merged_dataframe['BOK2016']['close_price']

In [ ]:


import matplotlib.pyplot as plt
plt.plot(range(len(aligned_dataframe.index)),aligned_dataframe['c1']['tr_dte'],
         range(len(aligned_dataframe.index)),aligned_dataframe['c2']['tr_dte'],
         range(len(aligned_dataframe.index)),aligned_dataframe['c3']['tr_dte'])
plt.grid()         
         
plt.show()

    

    

In [ ]:
merged_dataframe['c1']['tr_dte'].isnull().sum()
merged_dataframe['c3']['tr_dte'].isnull().sum()

In [ ]:
futures_dataframe

In [ ]:
print(trDte1)
print(trDte2)
print(trDte3)

In [ ]:
bf_price = (aligned_dataframe['c1']['close_price']+aligned_dataframe['c3']['close_price'])/2-aligned_dataframe['c2']['close_price']

In [ ]:
plt.plot(range(len(aligned_dataframe.index)),bf_price)
plt.grid()         
         
plt.show()

In [ ]:
import matplotlib.pyplot as plt
plt.plot(price_ratio)
plt.grid()
plt.show()

In [ ]:
import numpy as np
nan_matrix = np.empty((2,2))
nan_matrix[:] = np.NAN
nan_matrix